In [1]:
import json
import os
import numpy as np
import pandas as pd

In [37]:
train = pd.read_json('train.json', typ='frame', encoding='utf-8')
tag_song_map = pd.read_json('tag_song_pair.json', typ='frame', orient='table', encoding='utf-8')

tags = tag_song_map['tag']
tag_plylst_map = train[['id', 'tags']]

In [32]:
tags

0              
1            00
2           007
3        007시리즈
4           00s
          ...  
29155        힙해
29156       힙힙힙
29157      힛뎀포크
29158       힛뎀폭
29159      힛뎀폭스
Name: tag, Length: 29160, dtype: object

In [38]:
tag_plylst_map

,id,tags
0,61281,[락]
1,10532,"[추억, 회상]"
2,76951,"[까페, 잔잔한]"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,27616,[댄스]
...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]"
115067,106976,[일렉]
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]"
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]"


In [39]:
tag_plylst_pair = pd.read_json('tag_plylst_pair.json', typ='frame', orient='table', encoding='utf-8')

# unnest tags
tag_plylst_pair_unnest = np.dstack(
    (
        np.repeat(tag_plylst_pair.tag.values, list(map(len, tag_plylst_pair.plylst_id))), 
        np.concatenate(tag_plylst_pair.plylst_id.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
tag_plylst_pair = pd.DataFrame(data = tag_plylst_pair_unnest[0], columns = tag_plylst_pair.columns)
tag_plylst_pair['tag'] = tag_plylst_pair['tag'].astype(str)

# unnest 객체 제거
del tag_plylst_pair_unnest

tag_plylst_pair

,tag,plylst_id
0,,95032
1,00,123921
2,00,38363
3,00,3324
4,00,81830
...,...,...
476326,힙힙힙,45778
476327,힛뎀포크,37691
476328,힛뎀폭,37691
476329,힛뎀폭스,37691


In [41]:
tags_similarity = pd.DataFrame(data=tags, columns=['tag'])
tags_similarity['tag_group'] = np.nan

In [42]:
tags_similarity

,tag,tag_group
0,,NaN
1,00,NaN
2,007,NaN
3,007시리즈,NaN
4,00s,NaN
...,...,...
29155,힙해,NaN
29156,힙힙힙,NaN
29157,힛뎀포크,NaN
29158,힛뎀폭,NaN


In [43]:
tags_similarity = tags_similarity.to_numpy()
tags = tags.to_numpy()

for i in range(len(tags)):
    tags_similarity[i, 1] = set()

tags_similarity

array([['', set()],
       ['00', set()],
       ['007', set()],
       ...,
       ['힛뎀포크', set()],
       ['힛뎀폭', set()],
       ['힛뎀폭스', set()]], dtype=object)

In [44]:
tags

0              
1            00
2           007
3        007시리즈
4           00s
          ...  
29155        힙해
29156       힙힙힙
29157      힛뎀포크
29158       힛뎀폭
29159      힛뎀폭스
Name: tag, Length: 29160, dtype: object

In [103]:
tag_list = tag_plylst_map[tag_plylst_map['id']==tag_plylst_pair.loc[50]['plylst_id']]

tag_list_unnest = np.dstack((
    np.repeat(tag_list.id.values, list(map(len, tag_list.tags))),
    np.concatenate(tag_list.tags.values))
)

tag_list = pd.DataFrame(data = tag_list_unnest[0], columns = tag_list.columns)
tag_list['id'] = tag_list['id'].astype(str)

del tag_list_unnest

tag_list

,id,tags
0,71635,락
1,71635,00년생
2,71635,버즈


In [104]:
a = set()

for i in tag_list['tags']:
    a.add(i)

print(a)

{'버즈', '락', '00년생'}


In [106]:
for i in range(len(tag_plylst_pair)):
    tag_index = np.where(tags==tag_plylst_pair.loc[i]['tag'])
    tag_list = tag_plylst_map[tag_plylst_map['id']==tag_plylst_pair.loc[i]['plylst_id']]
    tag_list_unnest = np.dstack(
        (
            np.repeat(tag_list.id.values, list(map(len, tag_list.tags))),
            np.concatenate(tag_list.tags.values)
        )
    )

    tag_list = pd.DataFrame(data = tag_list_unnest[0], columns = tag_list.columns)
    tag_list['id'] = tag_list['id'].astype(str)
    del tag_list_unnest
    for j in tag_list['tags']:
        tags_similarity[tag_index[0][0]][1].add(j)
    if(i%100==0): print(i)

0


In [ ]:
tag_group = pd.DataFrame(data=tags_similarity, columns=['tag', 'tag_group'])
tag_group.to_json('tag_group.json', orient='table')

tag_group